In [38]:

pip install BeautifulSoup4==4.8.2


Note: you may need to restart the kernel to use updated packages.


In [39]:
from bs4 import BeautifulSoup as bs
import urllib.request
import requests
import os
import pymysql
import uuid
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus
import re

In [40]:
juso_db = pymysql.connect(
    user='ssafy', 
    passwd='ssafy', 
    host='127.0.0.1', 
    db='eurekauser', 
    charset='utf8'
)

cursor = juso_db.cursor(pymysql.cursors.DictCursor)

In [41]:
def getanimalnum(pageNum):
    ##다운 받을 주소
    baseUrl = 'https://www.zooseyo.or.kr/Yu_board/freesale_ty_1.html?animal=%EA%B0%95%EC%95%84%EC%A7%80&area=&ty=1&page='   
    url = baseUrl+str(pageNum)  
    req=requests.get(url)
    print(req)
    html = req.text
    soup = bs(html, "html.parser")
    info=soup.select('a[href]')
    checkpoint=False
    ct=0;
    animalnum=[]
    for i in info:
        tags=i.get('href')
        if("sell_up.html?gort=g"==tags):
            ct+=1
        num=tags.split("&")[0].split("?")
        if(ct>1 and num[0]=="view.html"):
            animalnum.append(num[1].split("=")[1])
            
    return animalnum

In [42]:
def getinfo(animalNum):
    print("====개정보====")
    preUrl='https://www.zooseyo.or.kr/Yu_board/view.html?no='
    postUrl='&page=1&animal=&area='
    url=preUrl+animalNum+postUrl
    dogsinfo=[]
    
    req=requests.get(url)
    html = req.text
    soup = bs(html, "html.parser")

    #개번호 등록
    dogsinfo.append(animalNum)    
    
    #url 등록
    dogsinfo.append(url)

    
    ##img 경로등록
    imginfo=soup.select('img[src]')
    imglist=[]
    for i in imginfo:
        tags=i.get('src')
        if('/dog_sale/' in tags):
#             print(tags)
            imglist.append('https://www.zooseyo.or.kr'+tags)
    dogsinfo.append(imglist)

    ##개정보 등록
    infos=soup.get_text().split()
    
   ##날짜 인덱스 찾기
#     print(infos)
    for i in range(len(infos)):
        try:
            if"무료분양" in infos[i]:
                dogsinfo.append(infos[i].split("(")[1])
                break
        except:
            dogsinfo.append("")
    
    
    
    idx=0
    for idx in [356,364,368]:
        dogsinfo.append(infos[idx])
    

    
    
    return dogsinfo

In [43]:
def saveimg(directory,url,name="",dognum=""):
    if not name.isalnum() or ":" in name :
        return 
    if "믹스" in name or "MIX" in name.capitalize():
        name = "믹스"
    filename=name+"_"+str(dognum)
    n=1
    imguuid=uuid.uuid1();
    save_path= directory+"/"+name
    save_root = directory+"/"+name
    if not os.path.exists(save_root):os.makedirs(save_root)
    try:
        with urlopen(url) as f:
            with open(os.path.join(save_path, filename+"_"+str(imguuid)+'.jpg'),'wb') as h: # w - write b - binary
                img = f.read()
                h.write(img)
        print(f"폴더명 :  { name} url : {url} 파일명 : {filename}")
        return [filename+"_"+str(imguuid)+'.jpg']
    except:
        print("error")
    

In [44]:
def valiatae(data):
    return True

In [45]:
##데이터 추가 고민중
def saveAndInsert(dogInfos):
    return 

In [ ]:
if __name__ == "__main__":
    
    #크롤링 멈추는 날짜(등록일 기준)
    endDate="2020.03.15"  
    
    
    dogInfos=[]
    ##넘겨볼 페이지 갯수
    for pageNum in range(1):
        animalNums=getanimalnum(pageNum)
        for num in animalNums:
            info=getinfo(num)
            #print("pageNum : ",pageNum,info)
            if(info[len(info)-4] in endDate):
                break
            dogInfos.append(info)
    print("####"*20)
    print("####"*20)
    print(f'Data Cnt: {len(dogInfos)}')
    print("####"*20)
    print("####"*20)
    
    
    
    
        
    

<Response [200]>
====개정보====
====개정보====
====개정보====
====개정보====
====개정보====
====개정보====
====개정보====


In [ ]:
for info in dogInfos:
    if(info[2] == []) :
        continue
    for img in info[2][1:]:
        ##요기에 원하는 경로
        saveimg("C:/Users/sskim/Downloads/Dogs",img,info[len(info)-3],info[0])
    
            